In [ ]:
def feature_scale(data,response_index,ALPHA):
    import os
    import pandas as pd
    from scipy import stats as st
    import numpy as np

    z = list(data.columns)                              #getting the list of all variable in the dataframes
    res = list(data[z[response_index]].unique())        # pulling unique responses from the response variable to create subset
    dat = []
    for i in res:                                      #conditional subsetting the data on the basis of Response variable 
        elem ="data "+ str(i)                          # and saving the data subset as objects in the list 
        dat.append(elem)
        J=res.index(i)
        dat[J] = data[data[str(z[response_index])] == res[J]]
    Res = data[z[response_index]]                     # setting response variable for chi square testing
    Data = data.drop(z[response_index],axis=1)        # Removing Response variable from the data

    l=list(Data.columns)                             # getting list of all columns apart from response variable from data
    L=[]
    alpha=float(ALPHA)                               # setting level of significance as floating point
    for i in range(len(l)):    
        I=l[i]
        if np.dtype(Data[I]) != "O" and np.dtype(Data[I]) != "O" :  # conditioning over variable for qualitative if dtype is object
            for j in range(len(dat)):
                J=dat[j]
                if j<1:                                    # setting two variable for using two dataset simultaneously for test
                    K=dat[j+1]
                else:
                    continue                          # testing for normality using shapiro test
                p1 = st.shapiro(J[I])
                p2 = st.shapiro(K[I])
                if p1[1] > alpha and p2[1] > alpha:     # condition for parametric test if normality exist
                        x = list(st.ttest_ind_from_stats(J[I].mean(),J[I].std(),J[I].count(),
                                            K[I].mean(),K[I].std(),K[I].count()))  # indep sample t test for testing whether to keep 
                        if x[1] < alpha:                                                # the feature or not
                            if I not in L:
                                L.append(I)                 # if featuure is important for response variable append in the list
                else:
                    x = list(st.mannwhitneyu(J[I],K[I]))    # if the normality fails use manwhitneyu test which is non parametric version of t test
                    if x[1] < alpha:
                        if I not in L:
                            L.append(I)
            
        else:
            crostbs = pd.crosstab(Res,Data[I])              # for categorical variable we test for the association for 
            chi2, pchi, dof, expected = st.chi2_contingency(crostbs.values)   #all the variable with response
            if pchi < alpha:                                            # if association is significant append the variable in the list
                    if I not in L:
                        L.append(I)
    return(L)                                               # L will return the namee of all the variables that are important for model building